# Pretrained CIFAR-10 model
This is taken from https://github.com/ARM-software/CMSIS_5/tree/develop/CMSIS/NN/Examples/ARM/arm_nn_examples/cifar10

In [ ]:
import re
import numpy as np

# Grab parameters
These are in tensorflow-models/cmsis_pretrained/arm_nnexamples_cifar10_parameter.h

In [ ]:
CONV1_IM_DIM = 32
CONV1_IM_CH = 3
CONV1_KER_DIM = 5
CONV1_PADDING = 2
CONV1_STRIDE = 1
CONV1_OUT_CH = 32
CONV1_OUT_DIM = 32

POOL1_KER_DIM = 3
POOL1_STRIDE = 2
POOL1_PADDING = 0
POOL1_OUT_DIM = 16

CONV2_IM_DIM = 16
CONV2_IM_CH = 32
CONV2_KER_DIM = 5
CONV2_PADDING = 2
CONV2_STRIDE = 1
CONV2_OUT_CH = 16
CONV2_OUT_DIM = 16

POOL2_KER_DIM = 3
POOL2_STRIDE = 2
POOL2_PADDING = 0
POOL2_OUT_DIM = 8

CONV3_IM_DIM = 8
CONV3_IM_CH = 16
CONV3_KER_DIM = 5
CONV3_PADDING = 2
CONV3_STRIDE = 1
CONV3_OUT_CH = 32
CONV3_OUT_DIM = 8

POOL3_KER_DIM = 3
POOL3_STRIDE = 2
POOL3_PADDING = 0
POOL3_OUT_DIM = 4

IP1_DIM = 4*4*32
IP1_IM_DIM = 4
IP1_IM_CH = 32
IP1_OUT = 10

In [ ]:
def parse_pretrained_data(fName, pretrained_data):
    with open(fName) as fp:
        for line in fp:
            if len(line) == 0 or line[0] != '#':
                continue
            line = line.split()
            param = line[1]
            data = line[2]
            if '{' in data:
                data = re.sub("[{}]", "", data)
                pretrained_data[param] = np.array(map(int, data.split(',')))
            else:
                pretrained_data[param] = int(data)


In [ ]:
ptd = {}
parse_pretrained_data("cmsis_pretrained/arm_nnexamples_cifar10_inputs.h", ptd)
parse_pretrained_data("cmsis_pretrained/arm_nnexamples_cifar10_weights.h", ptd)


### Reshape the data

In [ ]:
# Might need to make this 4D
ptd["IMG_DATA"] = ptd["IMG_DATA"].reshape((CONV1_IM_DIM, CONV1_IM_DIM, CONV1_IM_CH)).astype("uint8")
#https://stackoverflow.com/questions/36223157/set-weight-and-bias-tensors-of-tensorflow-conv2d-operation
ptd["CONV1_WT"] = ptd["CONV1_WT"].reshape((CONV1_KER_DIM, CONV1_KER_DIM, CONV1_IM_CH, CONV1_OUT_CH)).astype("int8")
ptd["CONV2_WT"] = ptd["CONV2_WT"].reshape((CONV2_KER_DIM, CONV2_KER_DIM, CONV1_OUT_CH, CONV2_OUT_CH)).astype("int8")
ptd["CONV3_WT"] = ptd["CONV3_WT"].reshape((CONV3_KER_DIM, CONV3_KER_DIM, CONV2_OUT_CH, CONV3_OUT_CH)).astype("int8")
ptd["IP1_WT"] = ptd["IP1_WT"].reshape((IP1_IM_DIM, IP1_IM_DIM, IP1_IM_CH, IP1_OUT)).astype("int8")
ptd["CONV1_BIAS"] = ptd["CONV1_BIAS"].astype("int8")
ptd["CONV2_BIAS"] = ptd["CONV2_BIAS"].astype("int8")
ptd["CONV3_BIAS"] = ptd["CONV3_BIAS"].astype("int8")
ptd["POOL1_KER_DIM"] = np.array([1, POOL1_KER_DIM, POOL1_KER_DIM, 1])
ptd["POOL2_KER_DIM"] = np.array([1, POOL2_KER_DIM, POOL2_KER_DIM, 1])
ptd["POOL3_KER_DIM"] = np.array([1, POOL3_KER_DIM, POOL3_KER_DIM, 1])
ptd["POOL1_STRIDE"] = np.array([1, POOL1_STRIDE, POOL1_STRIDE, 1])
ptd["POOL2_STRIDE"] = np.array([1, POOL2_STRIDE, POOL2_STRIDE, 1])
ptd["POOL3_STRIDE"] = np.array([1, POOL3_STRIDE, POOL3_STRIDE, 1])

In [ ]:
print(ptd.keys())
print(ptd["CONV1_BIAS"])

In [ ]:
def deepnn(input_image):
  with tf.name_scope("Layer1"):
    CONV1_WT = weight_variable(ptd['CONV1_WT'], name='CONV1_WT')
    b_fc1 = bias_variable(ptd['CONV1_BIAS'], name='CONV1_BIAS')
    conv_kernel_1 = tf.nn.conv2d(input_image, CONV1_WT, [1, CONV1_STRIDE, CONV1_STRIDE, 1], padding='VALID')
    bias_layer_1 = tf.nn.bias_add(conv_kernel_1, b_fc1)
    h_fc1 = tf.nn.relu(bias_layer_1)
    img_buffer2 = tf.nn.max_pool(h_fc1, ptd["POOL1_KER_DIM"], ptd["POOL1_STRIDE"], padding="SAME")

  with tf.name_scope("Layer2"):
    W_fc2 = weight_variable([128, 64], name='W_fc2')
    b_fc2 = bias_variable([64], name='b_fc2')
    a_fc2 = tf.add(tf.matmul(h_fc1, W_fc2), b_fc2, name="zscore")
    h_fc2 = tf.nn.relu(a_fc2)
  
  with tf.name_scope("OuputLayer"):
    W_fc3 = weight_variable([64, 10], name='W_fc3')
    b_fc3 = bias_variable([10], name='b_fc3')
    logits = tf.add(tf.matmul(h_fc2, W_fc3), b_fc3, name="logits")
    y_pred = tf.argmax(logits, 1, name='y_pred')

  return y_pred, logits